In [1]:
from pyspark.sql import SparkSession
from sparkmeasure import StageMetrics


spark = SparkSession.builder.appName('q6').config("spark.jars", "/home/athina/Downloads/spark-measure_2.12-0.17.jar").getOrCreate()
stagemetrics = StageMetrics(spark)


stagemetrics.begin()

df_mv = (spark.read.format('csv').option('header', 'true').option('infereSchema', 'true').load("movie.csv"))

#df_tag = spark.read.format('csv').option('header', 'true').option('infereSchema', 'true').load('/home/administrator/Downloads/movielens/tag.csv')
df_rt = spark.read.format('csv').option('header', 'true').option('inferSchema', 'true').load('rating.csv')


#--Q6 START--------------------------------------------------------------------------------------------------------
#df_rt.groupBy('movieId')
#df_rt.show(10)  20000263 distinct movies 26744

df_rt.select('movieId').distinct().count()

#want = df_rt.groupBy('movieId').sum('rating')

#df.select(countDistinct("Sales").alias("Distinct Sales")).show()
#df_rt.groupBy('movieId').count().select('movieId', df_rt.col('count').alias('num of ratings')).show()
#from pyspark.sql.functions import countDistinct
#gr = df_rt.groupBy("movieId").agg(countDistinct("rating"))
#gr.orderBy('count(rating)', ascending = False).show(5)

#q6
d= df_rt.groupBy('movieId').count()
d=d.withColumnRenamed('count', 'numOfRatings')
d= d.join(df_mv, df_mv.movieId == d.movieId, "inner").select(d.movieId, df_mv.title, d.numOfRatings)
d.orderBy('numOfRatings', ascending = False).show(5)

stagemetrics.end()

#print performance metrics
stagemetrics.print_report()
spark.stop()

#--Q6 END----------------------------------------------------------------------------------------------------------

+-------+--------------------+------------+
|movieId|               title|numOfRatings|
+-------+--------------------+------------+
|    296| Pulp Fiction (1994)|       67310|
|    356| Forrest Gump (1994)|       66172|
|    318|Shawshank Redempt...|       63366|
|    593|Silence of the La...|       63299|
|    480|Jurassic Park (1993)|       59715|
+-------+--------------------+------------+
only showing top 5 rows


Scheduling mode = FIFO
Spark Context default degree of parallelism = 2
Aggregated Spark stage metrics:
numStages => 9
numTasks => 422
elapsedTime => 74559 (1.2 min)
stageDuration => 72093 (1.2 min)
executorRunTime => 123212 (2.1 min)
executorCpuTime => 106679 (1.8 min)
executorDeserializeTime => 3642 (4 s)
executorDeserializeCpuTime => 2194 (2 s)
resultSerializationTime => 83 (83 ms)
jvmGCTime => 1761 (2 s)
shuffleFetchWaitTime => 0 (0 ms)
shuffleWriteTime => 726 (0.7 s)
resultSize => 1397020 (1364.0 KB)
diskBytesSpilled => 0 (0 Bytes)
memoryBytesSpilled => 0 (0 Bytes)
pe